In [16]:
departure_elements = driver.find_elements(By.XPATH, '//div[@data-testid="timeline_location_airport_departure"]')
arrival_elements = driver.find_elements(By.XPATH, '//div[@data-testid="timeline_location_airport_arrival"]')
print("bp5.1")
total_transit_element = driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_stops_0"]')[i]

if total_transit_element.text == 'Direct':
    print("bp5.2")                    
    routes= "None"
    departure_content = [element.text.split(" · ")[0] for element in departure_elements]
    arrival_content = [element.text.split(" · ")[0] for element in arrival_elements]
    truncate_routes = departure_content[:-1] + [arrival_content[0], arrival_content[-1]]
    routes = ", ".join(truncate_routes)
    print(routes)
else:
    print("bp5.3")                    

    departure_content = [element.text.split(" · ")[0] for element in departure_elements]
    arrival_content = [element.text.split(" · ")[0] for element in arrival_elements]
    truncate_routes = departure_content[:-1] + [arrival_content[-2], arrival_content[-1]]
    routes = ", ".join(truncate_routes)
    print(routes)                    
print("bp6")



bp5.1
bp5.2
DOH, DOH
bp6


In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.edge.options import Options
from datetime import datetime
import time
import random
import re


driver = webdriver.Edge()
driver.set_window_size(1366, 768)  # Set window size for consistent behavior

options = Options()
#options.add_argument("--headless")

job_id = 1
from_ = "CGK"
to = "DOH"
cabin_class = "ECONOMY"
date = "2023-08-01"
driver.get("https://flights.booking.com/flights/"+from_+".AIRPORT-"+to+".AIRPORT/?type=ONEWAY&adults=1&cabinClass="+cabin_class+"&children=&from="+from_+".AIRPORT&to="+to+".AIRPORT&fromCountry=&toCountry=&fromLocationName=Soekarno-Hatta+International+Airport&toLocationName=Changi+Airport&stops=0&depart="+date+"&sort=BEST&travelPurpose=leisure&aid=304142&label=gen173nr-1FEgdmbGlnaHRzKIICOOgHSDNYBGhoiAEBmAExuAEXyAEM2AEB6AEB-AELiAIBqAIDuAL4spmlBsACAdICJDU1MjlmNzk3LTY2ZGYtNDU3ZS05MGQ0LThiNjI5YmUwODA3MNgCBuACAQ")
data = []
c = 0

time.sleep(random.uniform(5,6))
search_button = driver.find_element(By.XPATH, '//button[@data-ui-name="button_search_submit"]')
search_button.click()
time.sleep(random.uniform(4,5))

list_airlines = driver.find_elements(By.XPATH, '//div[@class="css-4o3ibe"]')

try:
    pagination = driver.find_element(By.CLASS_NAME, 'Pagination-module__pages___wQykF')
    page_buttons = pagination.find_elements(By.TAG_NAME, 'button')
except Exception as e:
    for i in range(len(list_airlines)):
        try:
            c += 1
            flight_details = driver.find_elements(By.CSS_SELECTOR, 'button[data-testid="flight_card_bound_select_flight"]')
            
            if i >= len(flight_details):
                # Index out of range, break out of the loop
                break
            
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'button[data-testid="flight_card_bound_select_flight"]')))
            time.sleep(2)       
            flight_details[i].click()

            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[data-testid="timeline_leg_info_flight_number_and_class"]')))
            element = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="timeline_leg_info_flight_number_and_class"]')
            text = element.text
            flight_number = text.split(' · ')[0]
            flight_class = text.split(' · ')[1]

            flight_details_close = driver.find_element(By.XPATH, '//button[@aria-label="Close"]')
            flight_details_close.click()

            bandara_awal = driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_departure_airport_0"]')[i].text.split()[0]
            bandara_akhir = driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_destination_airport_0"]')[i].text.split()[-1]

            total_transit_element = driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_stops_0"]')[i]
            total_transit = 0 if total_transit_element.text == 'Direct' else int(re.search(r'\d+', total_transit_element.text).group())

            waktu_berangkat = pd.to_datetime(driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_departure_time_0"]')[i].text).time()
            waktu_tiba = pd.to_datetime(driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_destination_time_0"]')[i].text).time()
            total_waktu = ' '.join(driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_duration_0"]')[i].text.split()[:2])

            harga_element = driver.find_elements(By.XPATH, '//div[@class="css-vxcmzt"]')[i]
            harga = float(''.join(filter(lambda x: x.isdigit() or x == ',', harga_element.text)).replace(',', '.'))

            data.append({
                'job_id': job_id,
                'platform': 'Booking.com',
                #'date': pd.to_datetime(date, dayfirst=True).date(),
                'date': "2023-08-05",
                'departure_airport': bandara_awal,
                'destination_airport': bandara_akhir,
                'flight_code': flight_number,
                'cabin_class': flight_class,
                'departure_time': waktu_berangkat,
                'arrival_time': waktu_tiba,
                'transit': total_transit,
                'flight_duration': total_waktu,
                'rates': harga
            })

            print(c)
        except Exception as e:
            print(e)
            break
    
last_page_button = page_buttons[-1]  # Get the last page button
last_page_label = last_page_button.get_attribute('aria-label').strip()  # Extract the page label
last_page_number = int(''.join(filter(str.isdigit, last_page_label)))  # Extract the last page number as an integer

# Loop over each page button
for page in range(last_page_number):
    # Perform scraping on each page
    for i in range(len(list_airlines)):
        try:
            c += 1
            flight_details = driver.find_elements(By.CSS_SELECTOR, 'button[data-testid="flight_card_bound_select_flight"]')
            
            if i >= len(flight_details):
                # Index out of range, break out of the loop
                break
            
            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'button[data-testid="flight_card_bound_select_flight"]')))
            time.sleep(random.uniform(3,5))       
            flight_details[i].click()

            WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[data-testid="timeline_leg_info_flight_number_and_class"]')))
            time.sleep(random.uniform(3,5))       
            time.sleep(2)       

            element = driver.find_element(By.CSS_SELECTOR, 'div[data-testid="timeline_leg_info_flight_number_and_class"]')
            text = element.text
            flight_number = text.split(' · ')[0]
            flight_class = text.split(' · ')[1]

            flight_details_close = driver.find_element(By.XPATH, '//button[@aria-label="Close"]')
            flight_details_close.click()

            flight_marketing = driver.find_elemennts(By.XPATH, '//div[@data-testid="flight_card_carrier_0"]')[i].text.split()[0]
            bandara_awal = driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_departure_airport_0"]')[i].text.split()[0]
            bandara_akhir = driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_destination_airport_0"]')[i].text.split()[-1]

            total_transit_element = driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_stops_0"]')[i]
            total_transit = 0 if total_transit_element.text == 'Direct' else int(re.search(r'\d+', total_transit_element.text).group())

            waktu_berangkat = pd.to_datetime(driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_departure_time_0"]')[i].text).time()
            waktu_tiba = pd.to_datetime(driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_destination_time_0"]')[i].text).time()
            total_waktu = ' '.join(driver.find_elements(By.XPATH, '//div[@data-testid="flight_card_segment_duration_0"]')[i].text.split()[:2])

            harga_element = driver.find_elements(By.XPATH, '//div[@class="css-vxcmzt"]')[i]
            harga = float(''.join(filter(lambda x: x.isdigit() or x == ',', harga_element.text)).replace(',', '.'))

            data.append({
                'job_id': job_id,
                'platform': 'Booking.com',
                #'date': pd.to_datetime(date, dayfirst=True).date(),
                'date': "2023-08-05",
                'departure_airport': bandara_awal,
                'destination_airport': bandara_akhir,
                'flight_marketing' :flight_marketing,
                'flight_code': flight_number,
                'cabin_class': flight_class,
                'departure_time': waktu_berangkat,
                'arrival_time': waktu_tiba,
                'transit': total_transit,
                'flight_duration': total_waktu,
                'rates': harga
            })

            print(c)
        except Exception as e:
            print(e)
            print(c)
            # Click the "Search again" button to handle the error
            search_again_button = driver.find_element(By.XPATH, '//button[@data-testid="Button" and text()="Search again"]')
            search_again_button.click()
            break
            
    next_button = driver.find_element(By.XPATH, '//button[@aria-label="Next"]')
    time.sleep(5)
    next_button.click()
    time.sleep(5)

